In [1]:
# gensim modules
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec

# numpy
import numpy

# random
from random import shuffle

# classifier
from sklearn.linear_model import LogisticRegression


Using TensorFlow backend.


In [84]:
import sys
import os
import codecs
import smart_open

if sys.version > '3':
    control_chars = [chr(0x85)]
else:
    control_chars = [unichr(0x85)]
    
def normalize_text(text):
    norm_text = text.lower()

    # Replace breaks with spaces
    norm_text = norm_text.replace('<br />', ' ')

    # Pad punctuation with spaces on both sides
    for char in ['.', '"', ',', '(', ')', '!', '?', ';', ':']:
        norm_text = norm_text.replace(char, ' ' + char + ' ')

    return norm_text

limit_read = 12500
def get_big_file(dif_path):
    temp = u''
    read_file_count = 0
    for file in os.listdir(dir_path) :
        if file.endswith(".txt") and read_file_count < limit_read :
            f = codecs.open(dir_path + '/' + file, "r")
            read_file_count += 1;
            t_clean = f.read()
            for c in control_chars:
                t_clean = t_clean.replace(c, ' ')
            temp += t_clean
            temp += "\n"

    temp_norm = normalize_text(temp)
    return temp_norm

folders_source = {'aclImdb/train/pos':'aclImdb/train-pos.txt', 'aclImdb/train/neg':'aclImdb/train-neg.txt', 'aclImdb/test/pos':'aclImdb/test-pos.txt', 'aclImdb/test/neg':'aclImdb/test-neg.txt'}
for dir_path, file_path in folders_source.items():
    file_result = get_big_file(dir_path)
    with smart_open.smart_open(file_path, "w") as f:
        f.write(file_result)
print('read complete')

read complete


In [85]:
class LabeledLineSentence(object):
    def __init__(self, sources):
        self.sources = sources
        
        flipped = {}
        
        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')
    
    def __iter__(self):
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    yield LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no])
    
    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    self.sentences.append(LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no]))
        return self.sentences
    
    def sentences_perm(self):
        shuffle(self.sentences)
        return self.sentences

In [86]:
sources = {'aclImdb/test-neg.txt':'TEST_NEG', 'aclImdb/test-pos.txt':'TEST_POS', 'aclImdb/train-neg.txt':'TRAIN_NEG', 'aclImdb/train-pos.txt':'TRAIN_POS'}

sentences = LabeledLineSentence(sources)
print('Loaded')

Loaded


In [87]:
# PV-DM w/concatenation - window=5 (both sides) approximates paper's 10-word total window size
# Doc2Vec(dm=1, dm_concat=1, size=100, window=5, negative=5, hs=0, min_count=2, workers=cores),
# PV-DBOW 
# Doc2Vec(dm=0, size=100, negative=5, hs=0, min_count=2, workers=cores),
# PV-DM w/average
# Doc2Vec(dm=1, dm_mean=1, size=100, window=10, negative=5, hs=0, min_count=2, workers=cores),



#model = Doc2Vec(min_count=1, window=10, size=100, sample=1e-4, negative=5, workers=8)
#PV-DBOW 
model = Doc2Vec(dm=0, min_count=1, window=10, size=100, sample=1e-4, negative=5,hs=0, workers=8)

model.build_vocab(sentences.to_array())
print('build vocab')
for epoch in range(15):
    print("before train", epoch)
    model.train(sentences.sentences_perm(),total_examples=model.corpus_count, epochs=epoch)

#model.save('./imdb.d2v')
model.save('./imdb_pv-dbow.d2v')


print('build doc2vec')



build vocab
before train 0
before train 1
before train 2
before train 3
before train 4
before train 5
before train 6
before train 7
before train 8
before train 9
before train 10
before train 11
before train 12
before train 13
before train 14
build doc2vec


In [88]:
#model = Doc2Vec.load('./imdb.d2v')
model = Doc2Vec.load('./imdb_pv-dbow.d2v')

In [89]:

len(model.docvecs)

50000

In [90]:
train_arrays = numpy.zeros((2*limit_read, 100))
train_labels = numpy.zeros(2*limit_read)

for i in range(limit_read):
    train_arrays[i] = model.docvecs[i]
    train_arrays[limit_read + i] = model.docvecs[limit_read + i]
    train_labels[i] = 1
    train_labels[limit_read + i] = 0

test_arrays = numpy.zeros((2*limit_read, 100))
test_labels = numpy.zeros(2*limit_read)

for i in range(limit_read):
    
    test_arrays[i] = model.docvecs[2*limit_read + i]
    test_arrays[limit_read + i] = model.docvecs[3*limit_read + i]
    test_labels[i] = 1
    test_labels[limit_read + i] = 0

In [91]:
classifier = LogisticRegression()
classifier.fit(train_arrays, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [92]:
classifier.score(test_arrays, test_labels)

0.88624000000000003